In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, time
from bokeh import events
from bokeh.core.properties import Enum
from bokeh.io import output_file, output_notebook, curdoc
from bokeh.plotting import figure, show
from bokeh.models import CustomJS, ColumnDataSource, LinearAxis, Range1d, Toggle, Slider, DatetimeTickFormatter, Select, Tabs, TabPanel, HoverTool, CrosshairTool, Span, Div, RadioGroup, LabelSet
from bokeh.layouts import row, column, gridplot, layout
from bokeh_realtime_packagev1.functions import *
from bokeh_realtime_packagev1.conditions import *
output_file('layout2_v003.html')  # Render to static HTML, or
output_notebook()  # Render inline in a Jupyter Notebook
#from icecream import ic
from iertools.tb import TB
#pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns',None)
from dateutil.parser import parse
import random

Loading BokehJS ...

## Generando los datos de sonido en dB
Tomamos como referencia los datos del DTHIS del auditorio los modificamos con un random para obtener una serie de datos para cada espacio, descartando el espacio de Atención COFI por cuestiones de confidencialidad.

In [2]:
tmp0= TB(config_file='config_co2.ini',device_name='DTHIS')
sensor = "Nivel Sonoro"

fecha1 = parse("2022-02-02")
#fecha2 = fecha1+pd.Timedelta('1D')
termopares = tmp0.get_df(key=sensor,start_datetime=fecha1)
fecha1 = parse("2022-04-06 19:47:30")
termopares2 = tmp0.get_df(key=sensor,start_datetime=fecha1)
temp = pd.concat([termopares,termopares2])
sonido = [temp['Nivel Sonoro'][i] for i in range(len(temp))]
fechas=[]
for i in range(143):
    fechas.append(parse('2019-01-01 00:10:00')+(pd.Timedelta('10min')*i))
fechas.append(parse('2019-01-01 23:59:59'))
joining = pd.DataFrame()
zones = ['N1AU401', 'N1AU402', 'N1AU403', 'N1AU404', 'N2AU101', 'N2AU102', 'N2AU103', 'N2AU201', 'N2AU202', 'N2AU203', 'PBADM', 'PBCOORCOFI', 'PBCOORDINACION', 'PBSJUNTAS', 'PBSMAESTROS', 'PBVESTIBULO', 'N1LAB', 'N2LAB1', 'N2LAB2', 'PBCAFE']
for zone in zones:
    tmp=[]
    for i in range(144):
        tmp.append(sonido[i]+np.random.uniform(0,20))
    joining['dB_'+zone]=tmp
joining['ts']=fechas
joining['time']=fechas
joining.set_index('ts',inplace=True)

["CO2","HR","Humedad Relativa","I","Iluminacion","NPS","Nivel Sonoro","Promedio Pixeles","T","Temperatura","avg","ilum","lpx","max","min","stedv"]


## Generando los datos de CO2
Seguimos un procedimiento similar al anterior, excluyendo de misma manera el espacio de Atención COFI

In [3]:
tmp0= TB(config_file='config_co2.ini',device_name='ESP')
sensor = "s8lp"
fecha2=parse('2023-03-28 06:00:00')
fecha1=fecha2-pd.Timedelta('1D')+pd.Timedelta('10min')
termopares = tmp0.get_df(key=sensor,start_datetime=fecha1,end_datetime=fecha2).resample('10min').mean()
tmp2=[termopares['s8lp'][i] for i in range(143)]
tmp2.append(422.4)
zones = ['N1AU401', 'N1AU402', 'N1AU403', 'N1AU404', 'N2AU101', 'N2AU102', 'N2AU103', 'N2AU201', 'N2AU202', 'N2AU203', 'PBADM', 'PBCOORCOFI', 'PBCOORDINACION', 'PBSJUNTAS', 'PBSMAESTROS', 'PBVESTIBULO', 'N1LAB', 'N2LAB1', 'N2LAB2', 'PBCAFE']
for zone in zones:
    tmp=[]
    for i in range(144):
        tmp.append(tmp2[i]+np.random.uniform(0,10))
    joining['co2_'+zone]=tmp

["s8lp","user-agent"]


In [4]:
def do_numbers(n):
    x = np.random.uniform(0,100,n)
    x_int = [int(num) for num in x]
    x_int = [round(num/sum(x_int)*100) for num in x_int]
    if sum(x_int) != 100:
        return do_numbers(n)
    else:
        return x_int

In [5]:
for zone in zones:
    tmp1=[]
    tmp2=[]
    tmp3=[]
    for i in range(144):
        x = do_numbers(3)
        tmp1.append(x[0])
        tmp2.append(x[1])
        tmp3.append(x[2])
    joining['ilu_sub_'+zone]=tmp1
    joining['ilu_u_'+zone]=tmp2
    joining['ilu_sob_'+zone]=tmp3
occupation = joining.copy()

/tmp/ipykernel_17539/713927309.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  joining['ilu_sob_'+zone]=tmp3


In [6]:
occupation.columns.values

array(['dB_N1AU401', 'dB_N1AU402', 'dB_N1AU403', 'dB_N1AU404',
       'dB_N2AU101', 'dB_N2AU102', 'dB_N2AU103', 'dB_N2AU201',
       'dB_N2AU202', 'dB_N2AU203', 'dB_PBADM', 'dB_PBCOORCOFI',
       'dB_PBCOORDINACION', 'dB_PBSJUNTAS', 'dB_PBSMAESTROS',
       'dB_PBVESTIBULO', 'dB_N1LAB', 'dB_N2LAB1', 'dB_N2LAB2',
       'dB_PBCAFE', 'time', 'co2_N1AU401', 'co2_N1AU402', 'co2_N1AU403',
       'co2_N1AU404', 'co2_N2AU101', 'co2_N2AU102', 'co2_N2AU103',
       'co2_N2AU201', 'co2_N2AU202', 'co2_N2AU203', 'co2_PBADM',
       'co2_PBCOORCOFI', 'co2_PBCOORDINACION', 'co2_PBSJUNTAS',
       'co2_PBSMAESTROS', 'co2_PBVESTIBULO', 'co2_N1LAB', 'co2_N2LAB1',
       'co2_N2LAB2', 'co2_PBCAFE', 'ilu_sub_N1AU401', 'ilu_u_N1AU401',
       'ilu_sob_N1AU401', 'ilu_sub_N1AU402', 'ilu_u_N1AU402',
       'ilu_sob_N1AU402', 'ilu_sub_N1AU403', 'ilu_u_N1AU403',
       'ilu_sob_N1AU403', 'ilu_sub_N1AU404', 'ilu_u_N1AU404',
       'ilu_sob_N1AU404', 'ilu_sub_N2AU101', 'ilu_u_N2AU101',
       'ilu_sob_N2AU101',

In [7]:
dB_zones_au = []
dB_zones_others = []
for zone in occupation.columns:
    if 'dB_' in zone:
        if 'AU' in zone:
            dB_zones_au.append(zone)
        else:
            dB_zones_others.append(zone)
co2_zones=[]
co2_zones_au = []
co2_zones_others = []
for zone in occupation.columns:
    if 'co2_' in zone:
        if 'AU' in zone:
            co2_zones_au.append(zone)
        else:
            co2_zones_others.append(zone)
        co2_zones.append(zone)
sub_zones_au = []
sub_zones_others = []
for zone in occupation.columns:
    if 'ilu_sub_' in zone:
        if 'AU' in zone:
            sub_zones_au.append(zone)
        else:
            sub_zones_others.append(zone)
u_zones_au = []
u_zones_others = []
for zone in occupation.columns:
    if 'ilu_u_' in zone:
        if 'AU' in zone:
            u_zones_au.append(zone)
        else:
            u_zones_others.append(zone)
sob_zones_au = []
sob_zones_others = []
for zone in occupation.columns:
    if 'ilu_sob_' in zone:
        if 'AU' in zone:
            sob_zones_au.append(zone)
        else:
            sob_zones_others.append(zone)

In [8]:
co2_zones

['co2_N1AU401',
 'co2_N1AU402',
 'co2_N1AU403',
 'co2_N1AU404',
 'co2_N2AU101',
 'co2_N2AU102',
 'co2_N2AU103',
 'co2_N2AU201',
 'co2_N2AU202',
 'co2_N2AU203',
 'co2_PBADM',
 'co2_PBCOORCOFI',
 'co2_PBCOORDINACION',
 'co2_PBSJUNTAS',
 'co2_PBSMAESTROS',
 'co2_PBVESTIBULO',
 'co2_N1LAB',
 'co2_N2LAB1',
 'co2_N2LAB2',
 'co2_PBCAFE']

In [9]:
aulas_n = ['N1AU401', 'N1AU402', 'N1AU403', 'N1AU404', 'N2AU101', 'N2AU102', 'N2AU103', 'N2AU201', 'N2AU202', 'N2AU203']
others_n = ['PBADM', 'PBCOORCOFI', 'PBCOORDINACION', 'PBSJUNTAS', 'PBSMAESTROS', 'PBVESTIBULO', 'N1LAB', 'N2LAB1', 'N2LAB2', 'PBCAFE']

co2s_au=[occupation[name][100] for name in co2_zones_au]
dbs_au=[occupation[name][100] for name in dB_zones_au]
subs_au=[occupation[name][100] for name in sub_zones_au]
us_au=[occupation[name][100] for name in u_zones_au]
sobs_au=[occupation[name][100] for name in sob_zones_au]

co2s_others=[occupation[name][100] for name in co2_zones_others]
dbs_others=[occupation[name][100] for name in dB_zones_others]
subs_others=[occupation[name][100] for name in sub_zones_others]
us_others=[occupation[name][100] for name in u_zones_others]
sobs_others=[occupation[name][100] for name in sob_zones_others]

In [13]:
co2s_au

[460.1604518146597,
 453.30012443234915,
 456.3201039938855,
 453.648542014557,
 454.0449559743981,
 454.4552772835114,
 461.70163639728,
 457.2008614431004,
 458.5286392612756,
 458.30466139206953]

In [11]:
source=ColumnDataSource(occupation)
exceed_dB = np.arange(115,80,-5.)
warning_dB = np.arange(85,55,-5.)
adequate_dB = np.arange(60,-5,-5.)

exceed_co2 = np.arange(480,505,5.)
warning_co2 = np.arange(440,485,5.)
adequate_co2 = np.arange(400,445,5.)
p_o = figure(y_range=others_n, height=300,
           toolbar_location=None, tools="",width=603,margin=(0,0,0,0))

p_o.hbar(y=others_n, right=co2s_others,left=0, height=0.6,line_color='black',color='white')

p_o.xgrid.grid_line_color = None
p_o.x_range.start = 400
p_o.x_range.end=500
p_o.xaxis.axis_label= 'CO2 concentration [ppm]'
p_o.yaxis.major_label_text_align='right'
p_o.varea(x=exceed_co2,y1=0,y2=10,color='red',alpha=0.2)
p_o.varea(x=warning_co2,y1=0,y2=10,color='yellow',alpha=0.2)
p_o.varea(x=adequate_co2,y1=0,y2=10,color='green',alpha=0.2)

# p.yaxis.major_label_standoff = 100



p2_o = figure(y_range=others_n, height=300,width=553,
           toolbar_location=None,y_axis_location='right', tools="",margin=(0,0,0,15))

p2_o.hbar(y=others_n, right=dbs_others, height=0.6,line_color='black',color='white')

p2_o.xgrid.grid_line_color = None
p2_o.x_range.start = 115
p2_o.x_range.end=0
p2_o.xaxis.axis_label = 'Sound levels [dB]'
p2_o.yaxis.major_label_text_color='white'
p2_o.yaxis.major_label_standoff = 0
p2_o.yaxis.major_label_text_font_size = '0.5pt'
p2_o.varea(x=exceed_dB,y1=0,y2=10,color='red',alpha=0.2)
p2_o.varea(x=warning_dB,y1=0,y2=10,color='yellow',alpha=0.2)
p2_o.varea(x=adequate_dB,y1=0,y2=10,color='green',alpha=0.2)

p5_o = figure(y_range=aulas_n, height=300,width=550,
           toolbar_location=None, tools="",x_axis_type=None,x_range=p_o.x_range,margin=(0,0,0,55))

p5_o.hbar(y=aulas_n, right=co2s_au,left=0, height=0.6,line_color='black',color='white')

p5_o.xgrid.grid_line_color = None
# p5.x_range.start = 0
# p5.x_range.end=500

p5_o.yaxis.visible = True
p5_o.varea(x=exceed_co2,y1=0,y2=10,color='red',alpha=0.2)
p5_o.varea(x=warning_co2,y1=0,y2=10,color='yellow',alpha=0.2)
p5_o.varea(x=adequate_co2,y1=0,y2=10,color='green',alpha=0.2)

p6_o = figure(y_range=aulas_n, height=300,width=550,
           toolbar_location=None, tools="",y_axis_location='right',x_axis_type=None,x_range=p2_o.x_range,margin=(0,0,0,15))

p6_o.hbar(y=aulas_n, left=dbs_au, right=0, height=0.6,line_color='black',color='white')

p6_o.xgrid.grid_line_color = None
p6_o.x_range.start = 115
p6_o.x_range.end=0
p6_o.yaxis.major_label_text_color = 'white'
p6_o.yaxis.major_label_standoff = 0
p6_o.yaxis.major_label_text_font_size = '0.5pt'
p6_o.varea(x=exceed_dB,y1=0,y2=10,color='red',alpha=0.2)
p6_o.varea(x=warning_dB,y1=0,y2=10,color='yellow',alpha=0.2)
p6_o.varea(x=adequate_dB,y1=0,y2=10,color='green',alpha=0.2)


p3_o = figure(width=1915,height=306,x_axis_type='datetime')
for zona in co2_zones:
    if 'AU' in zona:
        p3_o.line(y=zona,x='time',source=source,line_color='blue',name=zona)
    elif 'LAB' in zona:
        p3_o.line(y=zona,x='time',source=source,line_color='red',name=zona)
    else:
        p3_o.line(y=zona,x='time',source=source,line_color='green',name=zona)
p3_o.yaxis.axis_label = 'CO2 concentration [ppm]'

aulas=pd.DataFrame()
aulas['subs']=subs_au
aulas['us']=us_au
aulas['sobs']=sobs_au
aulas['zones']=aulas_n
ilus_au=['subs','us','sobs']
p7_o=figure(y_range=aulas_n, height=300,width=550,toolbar_location=None, tools="",x_axis_type=None,margin=(0,0,0,53))

p7_o.hbar_stack(ilus_au,y='zones',source=aulas,color=['green','blue','red'],line_color='black')
p7_o.x_range.start = 0
p7_o.x_range.end =100

others=pd.DataFrame()
others['subs']=subs_au
others['us']=us_au
others['sobs']=sobs_au
others['zones']=others_n
ilus_others=['subs','us','sobs']
p4_o=figure(y_range=others_n, height=300,width=603,toolbar_location=None, tools="",x_range=p7_o.x_range,margin=(0,0,0,0))

p4_o.hbar_stack(ilus_others,y='zones',source=others,color=['green','blue','red'],line_color='black')
p4_o.x_range.start = 0
p4_o.x_range.end = 100
p4_o.xaxis.axis_label = 'Percentage of surfaces [%]'

perce =[]
for i in range(10):
    perce.append(random.randint(0,100))

df=pd.DataFrame()
df['x']=np.ones(10)*0.5
df['y']=np.array([0.38,1.48,2.58,3.68,4.8,5.9,7,8.1,9.2,10.3])
df['text']=[str(num)+'%' for num in perce]
textdf=ColumnDataSource(df)

p8_o = figure(height=300,
           toolbar_location=None, tools="",width=40,margin=(0,0,0,0),y_range=Range1d(0,11))


p8_o.text(x='x',y='y',text='text',text_align="center",
        text_font_size="10px",
        text_font="arial",
        text_font_style="normal",source=textdf)
p8_o.yaxis.visible = False
p8_o.xaxis.visible = False
p8_o.grid.visible = False

p9_o = figure(height=300,
           toolbar_location=None, tools="",width=40,margin=(0,0,0,0),y_range=Range1d(0,11))

textdf=ColumnDataSource(df)
p9_o.text(x='x',y='y',text='text',text_align="center",
        text_font_size="10px",
        text_font="arial",
        text_font_style="normal",source=textdf)
p9_o.yaxis.visible = False
p9_o.xaxis.visible = False
p9_o.grid.visible = False

occupation_panel=layout([[p7_o,p6_o,p5_o,p9_o],[p4_o,p2_o,p_o,p8_o],[p3_o]])

show(occupation_panel)

In [12]:


show(p8)

NameError: name 'p8' is not defined

In [ ]:
perce